# Dokumentacja Optymalizatora v1 (`opty.py`)

## 1. Filozofia Działania: Optymalizacja Dyskretna
Moduł `opty.py` nie jest prostym skryptem uruchamiającym obliczenia, lecz **algorytmem heurystycznym przeszukiwania przestrzeni dyskretnej**. W inżynierii budowlanej nie możemy płynnie zmieniać wymiarów (np. grubości blachy 7.43 mm nie istnieją), musimy poruszać się po dostępnych katalogach wyrobów hutniczych.

### Cel Algorytmu
Znalezienie pary geometrycznej: `(Profil Walcowany, Płaskownik)`, która:
1.  **Spełnia Warunki SGN:** Wskaźnik Wytężenia $UR \le 1.0$ (Stateczność + Wytrzymałość).
2.  **Spełnia Warunki Geometrii:** Klasa przekroju $\le 3$ (unikamy klasy 4, by nie redukować przekroju efektywnego).
3.  **Minimalizuje Funkcję Celu:** Masa konstrukcji ($kg/m$).

---

## 2. Architektura Logiczna: "Smart Step Search"

Zamiast sprawdzać każdą możliwą kombinację (metoda *Brute Force*, która trwałaby godziny), algorytm stosuje strategię **"Inteligentnego Kroku" (Smart Step)**. Opiera się ona na inżynierskim założeniu, że **wraz ze wzrostem sztywności profilu głównego (UPE), zapotrzebowanie na wzmocnienie (grubość płaskownika) maleje lub pozostaje stałe.**

### Główne Fazy Procesu:
1.  **Inicjalizacja:** Załadowanie bazy materiałów i profili, sortowanie ich rosnąco po wysokości $h_c$.
2.  **Pamięć Procesu:** Algorytm "pamięta", jaka grubość płaskownika była optymalna dla poprzedniego, mniejszego profilu.
3.  **Lokalne Przeszukiwanie (Local Search):** Dla bieżącego profilu sprawdza otoczenie poprzedniego optimum, szukając najlżejszego rozwiązania.
4.  **Badanie Wrażliwości (Widening):** Dla znalezionego optimum próbuje "rozszerzyć" słup, aby zwiększyć sztywność $I_y$ bez dodawania masy.

---

## 3. Symulacja Krok po Kroku (Przykład Obliczeniowy)

Prześledźmy działanie algorytmu na konkretnym przykładzie przejścia z profilu **UPE140** na **UPE160**.

### Założenia początkowe symulacji:
* **Katalog Płaskowników ($t_p$):** `[..., 6, 8, 10, 12, 15, ...]` mm.
* **Konfiguracja:** `START_SEARCH_OFFSET = 1` (zaczynamy 1 oczko wyżej niż poprzednio).

#### KROK A: Zakończono analizę UPE140
Załóżmy, że dla profilu **UPE140** algorytm znalazł, że najcieńszym bezpiecznym płaskownikiem jest **8 mm**.
* *Zapamiętany stan:* `indeks_optimum_poprzedni` wskazuje na **8 mm**.

#### KROK B: Rozpoczęcie analizy UPE160
Algorytm pobiera profil **UPE160**. Jest on sztywniejszy od UPE140, więc spodziewamy się, że płaskownik 8 mm (lub nawet cieńszy) powinien wystarczyć.

1.  **Ustalenie Punktu Startu:**
    * Poprzednie optimum: **8 mm**.
    * Offset w górę (+1 indeks): **10 mm**.
    * *Decyzja:* Zaczynamy przeszukiwanie od grubości **10 mm** w dół. (Pomijamy sprawdzanie grubości 12, 15, 20 mm – oszczędność czasu).

2.  **Szukanie Dna (Minimum Feasible Thickness):**
    Iterujemy w dół po liście `[10, 8, 6, 5...]`:
    * **Test 1 (10 mm):**
        * Solver liczy UPE160 + Płaskownik 10 mm.
        * Wynik: $UR = 0.65$ (Bardzo bezpiecznie).
        * *Akcja:* Zapamiętujemy jako kandydata, idziemy niżej.
    * **Test 2 (8 mm):**
        * Solver liczy UPE160 + Płaskownik 8 mm.
        * Wynik: $UR = 0.85$ (OK).
        * *Akcja:* Nowe minimum, idziemy niżej.
    * **Test 3 (6 mm):**
        * Solver liczy UPE160 + Płaskownik 6 mm.
        * Wynik: $UR = 1.05$ (**PRZEKROCZENIE**).
        * *Akcja:* **STOP**. Płaskownik 6 mm jest za słaby.
    
    **Wniosek:** Dla UPE160 minimalną bezpieczną grubością jest **8 mm** (nazwijmy to $t_{min}$).

#### KROK C: Analiza Wariantów (Raportowanie)
Algorytm bierze znalezione $t_{min}$ (8 mm) oraz kilka grubości w górę (zgodnie z `ILE_KROKOW_W_GORE`, np. +1 krok, czyli 10 mm) i wykonuje pełną analizę.

**Dla grubości 8 mm:**
1.  **Wariant MIN (Minimalne Otwarcie):**
    * Liczy dla $b_{otw} = 50 mm$.
    * Zapisuje wynik: "Masa: 22 kg/m, UR: 0.85".
2.  **Wariant MAX (Poszerzanie - Widening):**
    * Algorytm próbuje zwiększyć szerokość otwarcia (rozstaw ceowników), co zwiększa moment bezwładności $I_y$ (sztywność boczną) przy tej samej masie.
    * $b_{otw} = 55 mm \rightarrow UR = 0.84$ (OK).
    * $b_{otw} = 60 mm \rightarrow UR = 0.83$ (OK).
    * ...
    * $b_{otw} = 150 mm \rightarrow$ Stopka wchodzi w Klasę 4 (ścianka zbyt smukła) lub $UR$ rośnie przez lokalne efekty.
    * *Wynik:* Znaleziono maksymalne bezpieczne otwarcie np. 145 mm. Zapisuje wariant MAX.

#### KROK D: Globalny Bezpiecznik (Stop Condition)
Algorytm porównuje masę optymalnego UPE160 (z płaskownikiem 8 mm) z masą optymalnego UPE140 (z płaskownikiem 8 mm).
* Masa UPE140+8mm $\approx$ 18 kg/m.
* Masa UPE160+8mm $\approx$ 22 kg/m.
* *Wniosek:* Masa wzrosła. Jeśli taka sytuacja powtórzy się dla kolejnych 3 profili (UPE180, UPE200...), algorytm przerwie pracę, uznając, że optimum globalne zostało już minięte przy mniejszych profilach.

---

## 4. Parametry Sterujące Algorytmem (`config.py`)

Zrozumienie tych parametrów jest kluczowe dla sterowania precyzją i czasem obliczeń.

| Parametr | Wartość Typowa | Opis Funkcjonalny |
| :--- | :--- | :--- |
| `START_SEARCH_OFFSET` | `1` lub `2` | **Bufory bezpieczeństwa przy starcie.** Jeśli ustawimy 0, algorytm zacznie szukanie dokładnie od grubości z poprzedniego profilu. Jeśli jednak nowy profil ma inną geometrię (np. węższą stopkę), ta grubość może być za mała. Offset +2 każe mu zacząć sprawdzanie od dwóch grubości wyżej, by na pewno "trafić" w bezpieczny obszar przed zejściem w dół. |
| `ILE_KROKOW_W_GORE` | `0` lub `1` | **Ile wariantów raportować.** Jeśli 0, raportuje tylko absolutnie najlżejszy (najcieńszy) płaskownik. Jeśli 1, zaraportuje najcieńszy ORAZ jeden stopień grubszy (który będzie cięższy, ale może mieć dużo mniejszy UR, co inżynier może woleć). |
| `LIMIT_POSZERZANIA` | `3.0` | **Limit geometryczny.** Określa, jak szeroki może być słup względem minimum (np. max 3 razy szerszy niż min). Zapobiega tworzeniu konstrukcji absurdalnie szerokich. |
| `MAX_N_WZROSTOW_WAGI` | `3` | **Czułość stopu.** Po ilu "nieudanych" (cięższych) profilach z rzędu przerwać symulację. Zapobiega liczeniu ogromnych profili, które na pewno będą za ciężkie. |

---

## 5. Struktura Danych Wynikowych

Moduł `zbieracz` gromadzi spłaszczone słowniki wyników. Każdy wiersz wyniku zawiera pełną historię obliczenia:

* **Dane Wejściowe (`Input_...`):** Co weszło do solvera (geometria, obciążenia).
* **Wyniki Główne (`Res_...`):** UR, Masa, Ugięcia.
* **Diagnostyka (`Calc_...`):** Pośrednie wartości, np. nośność wyboczeniowa $N_{b,Rd}$.
* **Metadane (`Raport_Etap`):** Znacznik, czy jest to wariant `MIN_GEO` (wąski) czy `MAX_GEO` (szeroki).

Dzięki funkcji `zapisz_wszystkie_formaty`, otrzymujemy:
1.  **HTML:** Szybki podgląd wizualny (zielone/czerwone statusy).
2.  **CSV:** Dane do analizy w Excelu (np. wykres Masa vs UR).
3.  **JSON:** Pełny zrzut do archiwizacji lub dalszego przetwarzania w Pythonie.

---

## 6. Potencjalne Problemy i Ograniczenia Algorytmu

Mimo zastosowania heurystyki, istnieją scenariusze brzegowe:

1.  **Lokalne Minima (Pułapka Offsetu):**
    Jeśli zmiana geometrii między profilami jest drastyczna (np. przejście z UPE na HEB - choć tu obsługujemy tylko ceowniki), a `OFFSET` jest za mały, algorytm może zacząć przeszukiwanie od grubości, która już jest za mała (niespełnia warunków) i natychmiast przerwać pętlę "w dół", nie znajdując żadnego rozwiązania, mimo że grubsze płaskowniki by pasowały.
    * *Rozwiązanie:* W razie braku wyników dla profilu, zwiększ `START_SEARCH_OFFSET`.

2.  **Optymalizacja Jednokryterialna:**
    Funkcja celu to **Masa ($kg/m$)**. Algorytm zawsze wybierze lżejszy profil UPE z grubszym płaskownikiem, niż cięższy UPE z cieńszym płaskownikiem (jeśli suma mas jest mniejsza). Nie uwzględnia kosztów spawania (grubsza spoina = większy koszt robocizny).

3.  **Dyskretność Katalogu:**
    Rozwiązanie jest tak dobre, jak gęstość zdefiniowanej listy `dostepne_plaskowniki`. Jeśli w bazie mamy przeskok z 10mm na 15mm, optymalizator nie znajdzie idealnego 12mm, co może prowadzić do przewymiarowania.